In [9]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter

batch_size = 1

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/home/liam/Documents/notebook/files', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=True)


In [37]:

class Spike1d(torch.nn.Module):
    def __init__(self, n_in,n_out, pot = 1, input_layer = False):
        super(Spike1d, self).__init__()
        self.potentials = Parameter(torch.zeros(n_out, dtype=torch.float),requires_grad=False)
        self.pot = pot
        self.input_layer = input_layer
        self.fc = nn.Linear(n_in,n_out,bias = False)
        
    def func(self, a):
        return 2*a
    
    #integral of func
    def func_int(self, a):
        return a**2
    
    def forward(self, x):
        if not self.input_layer:
            x = self.fc(x)
        
        #x has n rows, (n of input layer) columns
        
        if x.shape != self.potentials.shape:
            raise Exception("Size mismatch {} x {}".format(x.shape,self.potentials.shape))
        
        x = x.unsqueeze(-1)
        
        #calculate potential change
        delta_p = torch.sum(self.func_int(x),-1)
        self.potentials = Parameter(self.potentials + delta_p)
        
        #generate spikes
        spikes = (self.potentials > self.pot).float()
        self.potentials = Parameter(self.potentials - (spikes * self.pot))
        print(self.potentials)
        
        return spikes
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.spike1 = Spike1d(2,2, input_layer = True)
        self.spike2 = Spike1d(2,5)
        self.spike3 = Spike1d(5,2)
    def forward(self, x):
        x = self.spike1(x)
        print(x)
        print()
        #x = self.fc1(x)
        x = self.spike2(x)
        print(x)
        print()
        
        #x = self.fc2(x)#.unsqueeze(-1)
        x = self.spike3(x)
        print(x)
        return
#network = Net()

#out = network(torch.tensor([0.3]*2, dtype=torch.float))#torch.randn(1, dtype=torch.float))

In [38]:
model = Net()

for i in range(15):

    out = model(torch.tensor([2**(1/2)]*2, dtype=torch.float))#torch.randn(1, dtype=torch.float))
    print("#########")
    

Parameter containing:
tensor([1.0000, 1.0000], requires_grad=True)
tensor([1., 1.])

Parameter containing:
tensor([0.1969, 0.0572, 0.8639, 0.2104, 0.0989], requires_grad=True)
tensor([0., 0., 0., 0., 0.])

Parameter containing:
tensor([0., 0.], requires_grad=True)
tensor([0., 0.])
#########
Parameter containing:
tensor([2.0000, 2.0000], requires_grad=True)
tensor([1., 1.])

Parameter containing:
tensor([0.3939, 0.1144, 0.7278, 0.4208, 0.1977], requires_grad=True)
tensor([0., 0., 1., 0., 0.])

Parameter containing:
tensor([0.0024, 0.1335], requires_grad=True)
tensor([0., 0.])
#########
Parameter containing:
tensor([3.0000, 3.0000], requires_grad=True)
tensor([1., 1.])

Parameter containing:
tensor([0.5908, 0.1716, 0.5917, 0.6312, 0.2966], requires_grad=True)
tensor([0., 0., 1., 0., 0.])

Parameter containing:
tensor([0.0048, 0.2670], requires_grad=True)
tensor([0., 0.])
#########
Parameter containing:
tensor([4.0000, 4.0000], requires_grad=True)
tensor([1., 1.])

Parameter containing:
t

In [26]:

#model = Net()

for param in model.parameters():
    print("")
    print(param.data)
    
print("done")


tensor([0.0842, 0.2562])

tensor([[-0.5120,  0.6180],
        [ 0.0046, -0.6635]])

tensor([0.3334, 0.5146, 0.0057, 0.4267, 0.2015])

tensor([[-0.4813, -0.6667],
        [-0.1001, -0.4142],
        [ 0.5539, -0.0437],
        [-0.6857,  0.6896],
        [-0.6070, -0.2592]])

tensor([0.2086, 0.0023])

tensor([[-0.1533, -0.4175,  0.3246, -0.3034, -0.2160],
        [ 0.2052,  0.3859, -0.0647, -0.2528,  0.3775]])
done
